## SLAM

Simultaneous Localisation and Mapping techniques are used to autonomous navigate from an initial point to a target point avoiding obstacles. The first step is the mapping generation using speciffic LIDAR sensor.

For this purpose a speciffic turtlebot SLAM node has to be launched to generate the map.

<img src="./Images/1_slam.png">

### World generation

First of all, we have to generate the world. We will use the Building Editor:
- in a new a terminal open gazebo as superuser (sudo gazebo)
- select "Building Editor" in edit menu
- construct the world
- save in model folder with name world1
- exit Building Editor
- in gazebo insert the world1 model and place it in desired position. You can add other objects if needed
- save the final world in world folder with name world1.world

<img src="./Images/2_02_builder1.png">

<img src="./Images/2_03_builder2.png">

The turtlebot3 SLAM node is created in the "turtlebot3_slam" package installed in our ROS environment. The launch file "turtlebot3_slam.launch" executes this node to generate the map.

Let's create a new "l2_SLAM.launch" launch file including the execution of this "turtlebot3_slam.launch" file

In [ ]:
<launch>
    <!-- gazebo launch with empty world and loading the new model we have created-->
    <include file ="$(find gazebo_ros)/launch/empty_world.launch">
        <arg name="world_name" value="$(find autonomous_robots)/world/world1.world"/>   
    </include>
    <!-- uploading urdf robot model into parameter server -->
    <param name="robot_description" command="$(find xacro)/xacro --inorder $(find turtlebot3_description)/urdf/turtlebot3_waffle_pi.urdf.xacro"/>  
     <!-- from parameter server spawn model into gazebo  -->
    <node pkg="gazebo_ros" type="spawn_model" name="spawn_urdf" args="-urdf -model turtlebot3_waffle_pi -param robot_description">
    </node>      
    <!-- Running the robot moving node -->
    <node pkg="teleop_twist_keyboard" type="teleop_twist_keyboard.py" name="teleop_twist_keyboard">
    </node>
    <!-- Slam Node running for creating map -->
    <include file="$(find turtlebot3_slam)/launch/turtlebot3_slam.launch">
        <arg name="slam_methods" value="gmapping"/>
    </include>
</launch>

Execute the previous launch file:

roslaunch autonomous_robots l2_SLAM.launch

- Navigate with keyboard to generate the map
- To save the map in the home directory, type:

rosrun map_server map_saver

- copy the 2 files generated in the map folder

<img src="./Images/2_04_map.png">

## Localization

Here we will use the map generated and suply it to the EMCL node.

The EMCL node will provide us the provabilistic localisation of the robot in the map.


<img src="./Images/2_05_localization.png">

Let's create a launch file 

In [ ]:
<launch>
     <!-- gazebo launch with empty world and loading the new model we have created-->
    <include file ="$(find gazebo_ros)/launch/empty_world.launch">
        <arg name="world_name" value="$(find autonomous_robots)/world/world1.world"/>   
    </include>
    <!-- uploading urdf robot model into parameter server -->
    <param name="robot_description" command="$(find xacro)/xacro --inorder $(find turtlebot3_description)/urdf/turtlebot3_waffle_pi.urdf.xacro"/>  
    <!-- from parameter server spawn model into gazebo  -->
    <node pkg="gazebo_ros" type="spawn_model" name="spawn_urdf" args="-urdf -model turtlebot3_waffle_pi -param robot_description">
    </node>      
    <!-- Running the robot moving node -->
    <node pkg="teleop_twist_keyboard" type="teleop_twist_keyboard.py" name="teleop_twist_keyboard">
    </node>
    <!-- using the map we saved earlier -->
    <arg name="map_file" value="$(find autonomous_robots)/map/map.yaml"/>
    <!-- Map_server package is used to handle mapp we saved -->
    <node pkg="map_server" type="map_server" name="map_server" args="$(arg map_file)"/>
    <!-- Adaptive monto carlo node running -->
    <include file="$(find turtlebot3_navigation)/launch/amcl.launch"/>
    <!-- Launching Rviz with no saved configuration -->
    <node pkg="rviz" type="rviz" name="rviz" args="-d $(find autonomous_robots)/rviz/localization.rviz"/>
    <!-- publisher transform for all joint and thier inbetween relations -->
    <node pkg="robot_state_publisher" type="robot_state_publisher" name="robot_state_publisher">
    <param name ="publish_frequency" type=" double" value="50.0"/>
    </node>
</launch>

In [ ]:
roslaunch autonomous_robots l3_Localization.launch

<img src="./Images/2_06_localization2.png">